In [1]:
import requests
from datetime import datetime, timedelta
import pandas as pd
from influxdb import InfluxDBClient
import matplotlib.pyplot as plt
import plotly.offline as offline
import plotly.graph_objs as go
import numpy as np
import decimal
import math
import statistics

import tensorflow as tf
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
from tensorflow import keras
from keras.models import Model
from keras.layers import Dense, SimpleRNN, LSTM, GRU, RNN, Input, Flatten
from tensorflow.keras.layers import concatenate
import mlflow

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거
pd.set_option('mode.chained_assignment',  None) # SettingWithCopyWarning 제거

# 1. x, y의 resolution이 같은 경우_1

In [2]:
train_x12345 = np.array([[[x for _ in range(5)] for x in range(1, 15) if x % 7 == 0]])
train_y12 = np.array([[[x for _ in range(2)] for x in range(1, 15) if x % 7 == 0]])
test_x12345 = np.array([[[x for _ in range(5)] for x in range(15, 29) if x % 7 == 0]])
test_y12 = np.array([[[x for _ in range(2)] for x in range(15, 29) if x % 7 == 0]])

In [3]:
train_x12345

array([[[ 7,  7,  7,  7,  7],
        [14, 14, 14, 14, 14]]])

In [4]:
train_y12

array([[[ 7,  7],
        [14, 14]]])

In [5]:
test_x12345

array([[[21, 21, 21, 21, 21],
        [28, 28, 28, 28, 28]]])

In [6]:
test_y12

array([[[21, 21],
        [28, 28]]])

In [7]:
# 입력 정의
input = Input(shape=(2, 5))
x = LSTM(24, return_sequences=1)(input)

# 출력 레이어 정의
output = Dense(2)(x)

# 모델 정의
model = Model(inputs=input, outputs=output)

# 모델 컴파일
model.compile(optimizer='adam', loss='mse')

# 모델 학습
model.fit(train_x12345, train_y12, validation_data=(test_x12345, test_y12), epochs=400)


Epoch 1/400
1/1 [==============================] - 2s 2s/step - loss: 122.3981 - val_loss: 609.8460
Epoch 2/400
1/1 [==============================] - 0s 22ms/step - loss: 121.8953 - val_loss: 608.6031
Epoch 3/400
1/1 [==============================] - 0s 21ms/step - loss: 121.3878 - val_loss: 607.3545
Epoch 4/400
1/1 [==============================] - 0s 22ms/step - loss: 120.8762 - val_loss: 606.1075
Epoch 5/400
1/1 [==============================] - 0s 22ms/step - loss: 120.3615 - val_loss: 604.8687
Epoch 6/400
1/1 [==============================] - 0s 21ms/step - loss: 119.8442 - val_loss: 603.6441
Epoch 7/400
1/1 [==============================] - 0s 22ms/step - loss: 119.3250 - val_loss: 602.4387
Epoch 8/400
1/1 [==============================] - 0s 21ms/step - loss: 118.8043 - val_loss: 601.2562
Epoch 9/400
1/1 [==============================] - 0s 22ms/step - loss: 118.2827 - val_loss: 600.0994
Epoch 10/400
1/1 [==============================] - 0s 21ms/step - loss: 117.7604 - 

In [8]:
test_y12

array([[[21, 21],
        [28, 28]]])

In [9]:
model.predict(test_x12345)

1/1 [==============================] - 0s 299ms/step


array([[[ 8.431168,  8.280571],
        [11.181889, 10.546545]]], dtype=float32)

# 2. x, y의 resolution이 같은 경우_2

In [371]:
train_x12345 = np.array([[[x for _ in range(5)] for x in range(1, 15)]])
train_y12 = np.array([[[x for _ in range(2)] for x in range(1, 15)]])
test_x12345 = np.array([[[x for _ in range(5)] for x in range(15, 29)]])
test_y12 = np.array([[[x for _ in range(2)] for x in range(15, 29)]])

In [372]:
train_x12345

array([[[ 1,  1,  1,  1,  1],
        [ 2,  2,  2,  2,  2],
        [ 3,  3,  3,  3,  3],
        [ 4,  4,  4,  4,  4],
        [ 5,  5,  5,  5,  5],
        [ 6,  6,  6,  6,  6],
        [ 7,  7,  7,  7,  7],
        [ 8,  8,  8,  8,  8],
        [ 9,  9,  9,  9,  9],
        [10, 10, 10, 10, 10],
        [11, 11, 11, 11, 11],
        [12, 12, 12, 12, 12],
        [13, 13, 13, 13, 13],
        [14, 14, 14, 14, 14]]])

In [373]:
train_y12

array([[[ 1,  1],
        [ 2,  2],
        [ 3,  3],
        [ 4,  4],
        [ 5,  5],
        [ 6,  6],
        [ 7,  7],
        [ 8,  8],
        [ 9,  9],
        [10, 10],
        [11, 11],
        [12, 12],
        [13, 13],
        [14, 14]]])

In [374]:
test_x12345

array([[[15, 15, 15, 15, 15],
        [16, 16, 16, 16, 16],
        [17, 17, 17, 17, 17],
        [18, 18, 18, 18, 18],
        [19, 19, 19, 19, 19],
        [20, 20, 20, 20, 20],
        [21, 21, 21, 21, 21],
        [22, 22, 22, 22, 22],
        [23, 23, 23, 23, 23],
        [24, 24, 24, 24, 24],
        [25, 25, 25, 25, 25],
        [26, 26, 26, 26, 26],
        [27, 27, 27, 27, 27],
        [28, 28, 28, 28, 28]]])

In [375]:
test_y12

array([[[15, 15],
        [16, 16],
        [17, 17],
        [18, 18],
        [19, 19],
        [20, 20],
        [21, 21],
        [22, 22],
        [23, 23],
        [24, 24],
        [25, 25],
        [26, 26],
        [27, 27],
        [28, 28]]])

In [310]:
# 입력 정의
input = Input(shape=(14, 5))
x = LSTM(24, return_sequences=1)(input)

# 출력 레이어 정의
output = Dense(2)(x)

# 모델 정의
model = Model(inputs=input, outputs=output)

# 모델 컴파일
model.compile(optimizer='adam', loss='mse')

# 모델 학습
model.fit(train_x12345, train_y12, validation_data=(test_x12345, test_y12), epochs=400)

Epoch 1/400
1/1 [==============================] - 2s 2s/step - loss: 75.7559 - val_loss: 485.7766
Epoch 2/400
1/1 [==============================] - 0s 24ms/step - loss: 75.1323 - val_loss: 484.1418
Epoch 3/400
1/1 [==============================] - 0s 23ms/step - loss: 74.5361 - val_loss: 482.6996
Epoch 4/400
1/1 [==============================] - 0s 24ms/step - loss: 73.9683 - val_loss: 481.3711
Epoch 5/400
1/1 [==============================] - 0s 24ms/step - loss: 73.4264 - val_loss: 480.1196
Epoch 6/400
1/1 [==============================] - 0s 24ms/step - loss: 72.9057 - val_loss: 478.9092
Epoch 7/400
1/1 [==============================] - 0s 24ms/step - loss: 72.4010 - val_loss: 477.7009
Epoch 8/400
1/1 [==============================] - 0s 24ms/step - loss: 71.9083 - val_loss: 476.5201
Epoch 9/400
1/1 [==============================] - 0s 24ms/step - loss: 71.4254 - val_loss: 475.4402
Epoch 10/400
1/1 [==============================] - 0s 23ms/step - loss: 70.9518 - val_loss: 

In [311]:
test_y12

array([[[15, 15],
        [16, 16],
        [17, 17],
        [18, 18],
        [19, 19],
        [20, 20],
        [21, 21],
        [22, 22],
        [23, 23],
        [24, 24],
        [25, 25],
        [26, 26],
        [27, 27],
        [28, 28]]])

In [312]:
model.predict(test_x12345)

1/1 [==============================] - 0s 263ms/step


array([[[3.8011606, 4.551572 ],
        [5.7755527, 6.5237045],
        [6.9173994, 7.454275 ],
        [7.7421894, 8.017586 ],
        [8.301514 , 8.389264 ],
        [8.59775  , 8.607721 ],
        [8.731215 , 8.728495 ],
        [8.796171 , 8.8004055],
        [8.8343315, 8.847699 ],
        [8.860434 , 8.881035 ],
        [8.879847 , 8.905525 ],
        [8.894878 , 8.923979 ],
        [8.906738 , 8.938112 ],
        [8.916189 , 8.94906  ]]], dtype=float32)

# x, y의 resolution이 다른경우

# 3.1 concatenate

In [16]:
train_x123 = np.array([[[x for _ in range(3)] for x in range(1, 15)]])
train_x45 = np.array([[[x for _ in range(2)] for x in range(1, 15) if x % 7 == 0]])
train_y12 = np.array([[[x for _ in range(2)] for x in range(1, 15) if x % 7 == 0]])

test_x123 = np.array([[[x for _ in range(3)] for x in range(15, 29)]])
test_x45 = np.array([[[x for _ in range(2)] for x in range(15, 29) if x % 7 == 0]])
test_y12 = np.array([[[x for _ in range(2)] for x in range(15, 29) if x % 7 == 0]])

In [314]:
train_x123

array([[[ 1,  1,  1],
        [ 2,  2,  2],
        [ 3,  3,  3],
        [ 4,  4,  4],
        [ 5,  5,  5],
        [ 6,  6,  6],
        [ 7,  7,  7],
        [ 8,  8,  8],
        [ 9,  9,  9],
        [10, 10, 10],
        [11, 11, 11],
        [12, 12, 12],
        [13, 13, 13],
        [14, 14, 14]]])

In [315]:
train_x45

array([[[ 7,  7],
        [14, 14]]])

In [318]:
train_y12

array([[[ 7,  7],
        [14, 14]]])

In [325]:
test_x123

array([[[15, 15, 15],
        [16, 16, 16],
        [17, 17, 17],
        [18, 18, 18],
        [19, 19, 19],
        [20, 20, 20],
        [21, 21, 21],
        [22, 22, 22],
        [23, 23, 23],
        [24, 24, 24],
        [25, 25, 25],
        [26, 26, 26],
        [27, 27, 27],
        [28, 28, 28]]])

In [320]:
test_x45

array([[[21, 21],
        [28, 28]]])

In [321]:
test_y12

array([[[21, 21],
        [28, 28]]])

In [330]:
# 입력 정의
input1 = Input(shape=(14, 3))
x1 = LSTM(24, return_sequences=False)(input1)

input2 = Input(shape=(2, 2))
x2 = LSTM(24, return_sequences=False)(input2)

# 두 LSTM의 출력을 결합
x = concatenate([x1, x2], axis=-1)

# 출력 레이어 정의
output = Dense(2)(x)

# 모델 정의
model = Model(inputs=[input1, input2], outputs=output)

# 모델 컴파일
model.compile(optimizer='adam', loss='mse')

# 모델 학습
model.fit([train_x123, train_x45], train_y12, validation_data=([test_x123, test_x45], test_y12), epochs=400)

Epoch 1/400
1/1 [==============================] - 3s 3s/step - loss: 121.2996 - val_loss: 600.0775
Epoch 2/400
1/1 [==============================] - 0s 25ms/step - loss: 119.7989 - val_loss: 596.6786
Epoch 3/400
1/1 [==============================] - 0s 25ms/step - loss: 118.2957 - val_loss: 593.2363
Epoch 4/400
1/1 [==============================] - 0s 26ms/step - loss: 116.7856 - val_loss: 589.7654
Epoch 5/400
1/1 [==============================] - 0s 25ms/step - loss: 115.2625 - val_loss: 586.2834
Epoch 6/400
1/1 [==============================] - 0s 25ms/step - loss: 113.7224 - val_loss: 582.8143
Epoch 7/400
1/1 [==============================] - 0s 24ms/step - loss: 112.1652 - val_loss: 579.3826
Epoch 8/400
1/1 [==============================] - 0s 24ms/step - loss: 110.5937 - val_loss: 576.0112
Epoch 9/400
1/1 [==============================] - 0s 24ms/step - loss: 109.0124 - val_loss: 572.7191
Epoch 10/400
1/1 [==============================] - 0s 25ms/step - loss: 107.4257 - 

In [327]:
test_y12

array([[[21, 21],
        [28, 28]]])

In [343]:
model.predict([test_x123, test_x45])

1/1 [==============================] - 0s 450ms/step


array([[10.752798, 10.592884]], dtype=float32)

# 3.2 Dense

In [17]:
# 입력 정의
input1 = Input(shape=(14, 3))
x1 = LSTM(24, return_sequences=False)(input1)

input2 = Input(shape=(2, 2))
x2 = LSTM(4, return_sequences=False)(input2)

# 두 LSTM의 출력을 각 완전연결층으로 매핑하고 결합
map_x1 = Dense(14)(x1)
map_x2 = Dense(14)(x2)
x = concatenate([map_x1, map_x2], axis=-1)

# 출력 레이어 정의
output = Dense(2)(x)

# 모델 정의
model = Model(inputs=[input1, input2], outputs=output)

# 모델 컴파일
model.compile(optimizer='adam', loss='mse')

# 모델 학습
model.fit([train_x123, train_x45], train_y12, validation_data=([test_x123, test_x45], test_y12), epochs=400)

Epoch 1/400
1/1 [==============================] - 3s 3s/step - loss: 126.3481 - val_loss: 616.9334
Epoch 2/400
1/1 [==============================] - 0s 24ms/step - loss: 125.4242 - val_loss: 615.4877
Epoch 3/400
1/1 [==============================] - 0s 25ms/step - loss: 124.5132 - val_loss: 614.0476
Epoch 4/400
1/1 [==============================] - 0s 24ms/step - loss: 123.6137 - val_loss: 612.6096
Epoch 5/400
1/1 [==============================] - 0s 24ms/step - loss: 122.7242 - val_loss: 611.1705
Epoch 6/400
1/1 [==============================] - 0s 24ms/step - loss: 121.8436 - val_loss: 609.7272
Epoch 7/400
1/1 [==============================] - 0s 24ms/step - loss: 120.9717 - val_loss: 608.2771
Epoch 8/400
1/1 [==============================] - 0s 24ms/step - loss: 120.1085 - val_loss: 606.8180
Epoch 9/400
1/1 [==============================] - 0s 24ms/step - loss: 119.2536 - val_loss: 605.3482
Epoch 10/400
1/1 [==============================] - 0s 24ms/step - loss: 118.4056 - 

In [18]:
test_y12

array([[[21, 21],
        [28, 28]]])

In [19]:
model.predict([test_x123, test_x45])

1/1 [==============================] - 1s 628ms/step


array([[10.010726, 10.159857]], dtype=float32)

# 3.3 flatten

In [20]:
# 입력 정의
input1 = Input(shape=(14, 3))
x1 = LSTM(24, return_sequences=0)(input1)

input2 = Input(shape=(2, 2))
x2 = LSTM(4, return_sequences=0)(input2)

# 두 LSTM의 출력을 평탄화하고 그 결과를 결합
flat_x1 = Flatten()(x1)
flat_x2 = Flatten()(x2)
x = concatenate([flat_x1, flat_x2], axis=-1)

# 출력 레이어 정의
output = Dense(2)(x)

# 모델 정의
model = Model(inputs=[input1, input2], outputs=output)

# 모델 컴파일
model.compile(optimizer='adam', loss='mse')

# 모델 학습
model.fit([train_x123, train_x45], train_y12, validation_data=([test_x123, test_x45], test_y12), epochs=400)

Epoch 1/400
1/1 [==============================] - 3s 3s/step - loss: 119.2149 - val_loss: 585.7169
Epoch 2/400
1/1 [==============================] - 0s 25ms/step - loss: 118.0033 - val_loss: 584.1816
Epoch 3/400
1/1 [==============================] - 0s 24ms/step - loss: 116.8663 - val_loss: 582.6838
Epoch 4/400
1/1 [==============================] - 0s 24ms/step - loss: 115.8086 - val_loss: 581.2192
Epoch 5/400
1/1 [==============================] - 0s 24ms/step - loss: 114.8272 - val_loss: 579.7811
Epoch 6/400
1/1 [==============================] - 0s 24ms/step - loss: 113.9141 - val_loss: 578.3607
Epoch 7/400
1/1 [==============================] - 0s 24ms/step - loss: 113.0587 - val_loss: 576.9496
Epoch 8/400
1/1 [==============================] - 0s 24ms/step - loss: 112.2503 - val_loss: 575.5439
Epoch 9/400
1/1 [==============================] - 0s 25ms/step - loss: 111.4794 - val_loss: 574.1445
Epoch 10/400
1/1 [==============================] - 0s 25ms/step - loss: 110.7386 - 

In [369]:
test_y12

array([[[21, 21],
        [28, 28]]])

In [370]:
model.predict([test_x123, test_x45])

1/1 [==============================] - 0s 459ms/step


array([[9.638989, 9.928236]], dtype=float32)

# x, y의 resolution이 같은 경우_2에 다중 입력 모델을 설정

In [11]:
train_x123 = np.array([[[x for _ in range(3)] for x in range(1, 15)]])
train_x45 = np.array([[[x for _ in range(2)] for x in range(1, 15)]])
train_y12 = np.array([[[x for _ in range(2)] for x in range(1, 15)]])

test_x123 = np.array([[[x for _ in range(3)] for x in range(15, 29)]])
test_x45 = np.array([[[x for _ in range(2)] for x in range(15, 29)]])
test_y12 = np.array([[[x for _ in range(2)] for x in range(15, 29)]])

In [377]:
train_x123

array([[[ 1,  1,  1],
        [ 2,  2,  2],
        [ 3,  3,  3],
        [ 4,  4,  4],
        [ 5,  5,  5],
        [ 6,  6,  6],
        [ 7,  7,  7],
        [ 8,  8,  8],
        [ 9,  9,  9],
        [10, 10, 10],
        [11, 11, 11],
        [12, 12, 12],
        [13, 13, 13],
        [14, 14, 14]]])

In [378]:
train_x45

array([[[ 1,  1],
        [ 2,  2],
        [ 3,  3],
        [ 4,  4],
        [ 5,  5],
        [ 6,  6],
        [ 7,  7],
        [ 8,  8],
        [ 9,  9],
        [10, 10],
        [11, 11],
        [12, 12],
        [13, 13],
        [14, 14]]])

In [379]:
train_y12

array([[[ 1,  1],
        [ 2,  2],
        [ 3,  3],
        [ 4,  4],
        [ 5,  5],
        [ 6,  6],
        [ 7,  7],
        [ 8,  8],
        [ 9,  9],
        [10, 10],
        [11, 11],
        [12, 12],
        [13, 13],
        [14, 14]]])

In [380]:
test_x123

array([[[15, 15, 15],
        [16, 16, 16],
        [17, 17, 17],
        [18, 18, 18],
        [19, 19, 19],
        [20, 20, 20],
        [21, 21, 21],
        [22, 22, 22],
        [23, 23, 23],
        [24, 24, 24],
        [25, 25, 25],
        [26, 26, 26],
        [27, 27, 27],
        [28, 28, 28]]])

In [381]:
test_x45

array([[[15, 15],
        [16, 16],
        [17, 17],
        [18, 18],
        [19, 19],
        [20, 20],
        [21, 21],
        [22, 22],
        [23, 23],
        [24, 24],
        [25, 25],
        [26, 26],
        [27, 27],
        [28, 28]]])

In [382]:
test_y12

array([[[15, 15],
        [16, 16],
        [17, 17],
        [18, 18],
        [19, 19],
        [20, 20],
        [21, 21],
        [22, 22],
        [23, 23],
        [24, 24],
        [25, 25],
        [26, 26],
        [27, 27],
        [28, 28]]])

In [13]:
# 입력 정의
input1 = Input(shape=(14, 3))
x1 = LSTM(24, return_sequences=1)(input1)

input2 = Input(shape=(14, 2))
x2 = LSTM(24, return_sequences=1)(input2)

# 두 LSTM의 출력을 결합
x = concatenate([x1, x2], axis=-1)

# 출력 레이어 정의
output = Dense(2)(x)

# 모델 정의
model = Model(inputs=[input1, input2], outputs=output)

# 모델 컴파일
model.compile(optimizer='adam', loss='mse')

# 모델 학습
model.fit([train_x123, train_x45], train_y12, validation_data=([test_x123, test_x45], test_y12), epochs=400)

Epoch 1/400
1/1 [==============================] - 3s 3s/step - loss: 65.7775 - val_loss: 459.6101
Epoch 2/400
1/1 [==============================] - 0s 24ms/step - loss: 64.9262 - val_loss: 457.6999
Epoch 3/400
1/1 [==============================] - 0s 24ms/step - loss: 64.0850 - val_loss: 455.8183
Epoch 4/400
1/1 [==============================] - 0s 24ms/step - loss: 63.2515 - val_loss: 453.9472
Epoch 5/400
1/1 [==============================] - 0s 25ms/step - loss: 62.4230 - val_loss: 452.0728
Epoch 6/400
1/1 [==============================] - 0s 25ms/step - loss: 61.5968 - val_loss: 450.1855
Epoch 7/400
1/1 [==============================] - 0s 24ms/step - loss: 60.7705 - val_loss: 448.2788
Epoch 8/400
1/1 [==============================] - 0s 25ms/step - loss: 59.9420 - val_loss: 446.3488
Epoch 9/400
1/1 [==============================] - 0s 24ms/step - loss: 59.1101 - val_loss: 444.3933
Epoch 10/400
1/1 [==============================] - 0s 24ms/step - loss: 58.2736 - val_loss: 

In [387]:
test_y12

array([[[15, 15],
        [16, 16],
        [17, 17],
        [18, 18],
        [19, 19],
        [20, 20],
        [21, 21],
        [22, 22],
        [23, 23],
        [24, 24],
        [25, 25],
        [26, 26],
        [27, 27],
        [28, 28]]])

In [14]:
model.predict([test_x123, test_x45])

1/1 [==============================] - 0s 462ms/step


array([[[ 4.8262024,  5.938714 ],
        [ 8.105966 ,  9.200221 ],
        [ 9.97536  , 10.842083 ],
        [10.993781 , 11.732357 ],
        [11.518549 , 12.205347 ],
        [11.80319  , 12.4614105],
        [11.971035 , 12.608183 ],
        [12.075311 , 12.695444 ],
        [12.141145 , 12.747329 ],
        [12.182227 , 12.7770405],
        [12.206943 , 12.792599 ],
        [12.2207575, 12.799103 ],
        [12.227296 , 12.799736 ],
        [12.228923 , 12.796389 ]]], dtype=float32)

In [ ]:
# 입력 정의
input = Input(shape=(14, 5))
x = LSTM(24, return_sequences=1)(input)

# 출력 레이어 정의
output = Dense(2)(x)

# 모델 정의
model = Model(inputs=input, outputs=output)

# 모델 컴파일
model.compile(optimizer='adam', loss='mse')

# 모델 학습
model.fit(train_x12345, train_y12, validation_data=(test_x12345, test_y12), epochs=400)